# Tahap Pre Porcessing Data

Pada tahap ini dilakukan pra pemerosesan data untuk membersihkan data dari gangguan noise 

In [5]:
import pandas as pd 
import csv
import ast
import string 
import re #regex library
import preprocessor as p
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

[nltk_data] Downloading package punkt to C:\Users\RIDHO-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\RIDHO-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index


In [6]:
# membaca data hasil crawling dan labeling
data_twt = pd.read_csv("NewDataset/Dataset-label.csv")

In [7]:
data_twt

,tweet,label_sentimen
0,ASTAGAAAA BARU INGET SEKARANG SEKOLAH OFFLINE,-1
1,"Yg udh sekolah offline, lama lama bosan gsi se...",-1
2,@jinhwanchy Wah udah sekolah offline? Semangat...,1
3,"@Fajri_UN1TY Haii juga, yuk yg sekolah offline...",1
4,Sch! Hai gais hari ini sender PTM untuk yg per...,-1
...,...,...
2188,sekolah ku udah mulai offline dari maret dann ...,0
2189,mana udah offline sekolah,0
2190,gue kira senin bakal sekolah offline ternyata ...,0
2191,art! curhat dkit sender akhir2 ini capek bgt s...,1


## Case Folding

In [8]:
data_twt['tweet_cf'] = data_twt['tweet'].str.lower()

print('Case Folding Result : \n')
print(data_twt['tweet_cf'].head(5))
print('\n\n\n')

Case Folding Result : 

0        astagaaaa baru inget sekarang sekolah offline
1    yg udh sekolah offline, lama lama bosan gsi se...
2    @jinhwanchy wah udah sekolah offline? semangat...
3    @fajri_un1ty haii juga, yuk yg sekolah offline...
4    sch! hai gais hari ini sender ptm untuk yg per...
Name: tweet_cf, dtype: object






## Tokenizing

In [9]:
def remove_tweet_special(text):
    # menghapus tab, new line, dan back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # menghapus non ASCII (emotikon, tulisan china, dll)
    text = text.encode('ascii', 'replace').decode('ascii')
    # menghapus mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # menghapus URL yg tidak lengkap
    return text.replace("http://", " ").replace("https://", " ")
                
data_twt['tweet_cf'] = data_twt['tweet_cf'].apply(remove_tweet_special)

#menghapus nomor
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_twt['tweet_cf'] = data_twt['tweet_cf'].apply(remove_number)

#menghapus tanda baca
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_twt['tweet_cf'] = data_twt['tweet_cf'].apply(remove_punctuation)

#menghapus spasi awal dan akhir
def remove_whitespace_LT(text):
    return text.strip()

data_twt['tweet_cf'] = data_twt['tweet_cf'].apply(remove_whitespace_LT)

#menghapus spasi yg dobel menjadi satu spasi
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_twt['tweet_cf'] = data_twt['tweet_cf'].apply(remove_whitespace_multiple)

# menghapus karakter tunggal
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data_twt['tweet_clean'] = data_twt['tweet_cf'].apply(remove_singl_char)

def preprocessing_data(x):
    return p.clean(x)

data_twt['tweet_clean'] = data_twt['tweet_cf'].apply(preprocessing_data)

def tokenize_data(x):
    return p.tokenize(x)

data_twt['tweet_clean'] = data_twt['tweet_clean'].apply(tokenize_data)

In [12]:
alay_dict = pd.read_csv('kamus/kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

data_twt['tweet_clean_alay'] = data_twt['tweet_clean'].apply(normalize_alay) 
data_twt.drop_duplicates(keep=False,inplace=True)

In [13]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_twt['tweet_token'] = data_twt['tweet_clean_alay'].apply(word_tokenize_wrapper)

## Stopword Removal

In [14]:
from nltk.corpus import stopwords
print(set(stopwords.words('Indonesian')))

{'seringnya', 'begitu', 'meminta', 'berakhirlah', 'masalah', 'olehnya', 'malah', 'berapapun', 'memihak', 'semampunya', 'bersama', 'memerlukan', 'tunjuk', 'sebaik', 'boleh', 'memastikan', 'menginginkan', 'bilakah', 'kedua', 'baru', 'dilalui', 'dengan', 'sajalah', 'sebelumnya', 'tersebut', 'sejauh', 'bersama-sama', 'saat', 'kenapa', 'dibuatnya', 'mungkin', 'berkehendak', 'dimaksud', 'belum', 'berapalah', 'apa', 'sebaliknya', 'sementara', 'kan', 'menantikan', 'menurut', 'masa', 'sama-sama', 'akhir', 'rasanya', 'wahai', 'sebutnya', 'terus', 'awalnya', 'manalagi', 'semacam', 'sini', 'suatu', 'benarlah', 'berlalu', 'lanjutnya', 'berkata', 'buat', 'seingat', 'belakangan', 'bermula', 'tersebutlah', 'di', 'selama', 'tentang', 'tuturnya', 'ungkapnya', 'juga', 'katakanlah', 'dalam', 'tentu', 'adalah', 'justru', 'mengingatkan', 'pukul', 'sebenarnya', 'semakin', 'semula', 'memperlihatkan', 'ditunjuknya', 'terdahulu', 'ditunjuki', 'hendak', 'menanti', 'sampai-sampai', 'lah', 'berlangsung', 'sewaktu'

In [15]:
# list stopword indonesia dari library nltk
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

# menambah stopword manual
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'puan'])
len(list_stopwords)

# menambah stopword dari file luar
txt_stopword = pd.read_csv("kamus/stopword.csv", names= ["stopwords"], header = None)

# konversi stopword string ke list dan menambah stopword tambahan
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
len(list_stopwords)
# ---------------------------------------------------------------------------------------

# konversi list to dictionary
list_stopwords = set(list_stopwords)


#menghapus stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_twt['tweet_stopword'] = data_twt['tweet_token'].apply(stopwords_removal) 


print(data_twt['tweet_stopword'])

758
0                              [astaga, sekolah, offline]
1       [sekolah, offline, bosan, sekolah, offline, gu...
2                    [sekolah, offline, semangat, jojooo]
3       [unty, hai, yuk, sekolah, offline, kayak, sema...
4       [sch, hai, gais, sender, ptm, kali, sekolah, a...
                              ...                        
2188    [sekolah, ku, offline, maret, enak, offline, o...
2189                                   [offline, sekolah]
2190               [gue, senin, sekolah, offline, online]
2191    [art, curhat, dkit, sender, cape, banget, seko...
2192    [kelas, dibolehin, sekolah, offline, gue, kete...
Name: tweet_stopword, Length: 2193, dtype: object


In [16]:
# normalisasi kata

def normalisasi(tweet):
    kamus_slangword = eval(open("kamus/combined_slang_words.txt").read()) # Membuka dictionary slangword
    pattern = re.compile(r'\b( ' + '|'.join (kamus_slangword.keys())+r')\b') # Search pola kata (contoh kpn -> kapan)
    content = []
    for kata in tweet:
        filteredSlang = pattern.sub(lambda x: kamus_slangword[x.group()],kata) # Replace slangword berdasarkan pola review yg telah ditentukan
        content.append(filteredSlang.lower())
    tweet = content
    return tweet
data_twt['tweet_normalisasi'] = data_twt['tweet_stopword'].apply(normalisasi)
print(data_twt['tweet_normalisasi'])

0                              [astaga, sekolah, offline]
1       [sekolah, offline, bosan, sekolah, offline, sa...
2                    [sekolah, offline, semangat, jojooo]
3       [unty, hai, yuk, sekolah, offline, kayak, sema...
4       [sch, hai, teman teman, sender, ptm, kali, sek...
                              ...                        
2188    [sekolah, ku, offline, maret, enak, offline, d...
2189                                   [offline, sekolah]
2190              [saya, senin, sekolah, offline, daring]
2191    [art, curhat, dkit, sender, capek, banget, sek...
2192    [kelas, dibolehin, sekolah, offline, saya, ket...
Name: tweet_normalisasi, Length: 2193, dtype: object


##  Stemming

In [17]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_twt['tweet_normalisasi']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_twt['tweet_stemming'] = data_twt['tweet_normalisasi'].swifter.apply(get_stemmed_term)
print(data_twt['tweet_stemming'])

4767
------------------------


Pandas Apply:   0%|          | 0/2193 [00:00<?, ?it/s]

0                              [astaga, sekolah, offline]
1       [sekolah, offline, bosan, sekolah, offline, sa...
2                    [sekolah, offline, semangat, jojooo]
3       [unty, hai, yuk, sekolah, offline, kayak, sema...
4       [sch, hai, teman teman, sender, ptm, kali, sek...
                              ...                        
2188    [sekolah, ku, offline, maret, enak, offline, d...
2189                                   [offline, sekolah]
2190              [saya, senin, sekolah, offline, daring]
2191    [art, curhat, dkit, sender, capek, banget, sek...
2192    [kelas, dibolehin, sekolah, offline, saya, ket...
Name: tweet_stemming, Length: 2193, dtype: object


In [18]:
df = pd.DataFrame(data_twt)
df['tweet_stemming'] = df['tweet_stemming'].apply(str)

In [19]:
def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
data_twt["tweet_bersih"] = data_twt["tweet_stemming"].apply(join_text_list)

In [20]:
data_twt

,tweet,label_sentimen,tweet_cf,tweet_clean,tweet_clean_alay,tweet_token,tweet_stopword,tweet_normalisasi,tweet_stemming,tweet_bersih
0,ASTAGAAAA BARU INGET SEKARANG SEKOLAH OFFLINE,-1,astagaaaa baru inget sekarang sekolah offline,astagaaaa baru inget sekarang sekolah offline,astaga baru ingat sekarang sekolah offline,"[astaga, baru, ingat, sekarang, sekolah, offline]","[astaga, sekolah, offline]","[astaga, sekolah, offline]","['astaga', 'sekolah', 'offline']",astaga sekolah offline
1,"Yg udh sekolah offline, lama lama bosan gsi se...",-1,yg udh sekolah offline lama lama bosan gsi sek...,yg udh sekolah offline lama lama bosan gsi sek...,yang sudah sekolah offline lama lama bosan tid...,"[yang, sudah, sekolah, offline, lama, lama, bo...","[sekolah, offline, bosan, sekolah, offline, gu...","[sekolah, offline, bosan, sekolah, offline, sa...","['sekolah', 'offline', 'bosan', 'sekolah', 'of...",sekolah offline bosan sekolah offline saya bosan
2,@jinhwanchy Wah udah sekolah offline? Semangat...,1,wah udah sekolah offline semangat jojooo,wah udah sekolah offline semangat jojooo,wah sudah sekolah offline semangat jojooo,"[wah, sudah, sekolah, offline, semangat, jojooo]","[sekolah, offline, semangat, jojooo]","[sekolah, offline, semangat, jojooo]","['sekolah', 'offline', 'semangat', 'jojooo']",sekolah offline semangat jojooo
3,"@Fajri_UN1TY Haii juga, yuk yg sekolah offline...",1,unty haii juga yuk yg sekolah offline kayak ak...,unty haii juga yuk yg sekolah offline kayak ak...,unty hai juga yuk yang sekolah offline kayak a...,"[unty, hai, juga, yuk, yang, sekolah, offline,...","[unty, hai, yuk, sekolah, offline, kayak, sema...","[unty, hai, yuk, sekolah, offline, kayak, sema...","['unty', 'hai', 'yuk', 'sekolah', 'offline', '...",unty hai yuk sekolah offline kayak semangat aj...
4,Sch! Hai gais hari ini sender PTM untuk yg per...,-1,sch hai gais hari ini sender ptm untuk yg pert...,sch hai gais hari ini sender ptm untuk yg pert...,sch hai gais hari ini sender ptm untuk yang pe...,"[sch, hai, gais, hari, ini, sender, ptm, untuk...","[sch, hai, gais, sender, ptm, kali, sekolah, a...","[sch, hai, teman teman, sender, ptm, kali, sek...","['sch', 'hai', 'teman teman', 'sender', 'ptm',...",sch hai teman teman sender ptm kali sekolah al...
...,...,...,...,...,...,...,...,...,...,...
2188,sekolah ku udah mulai offline dari maret dann ...,0,sekolah ku udah mulai offline dari maret dann ...,sekolah ku udah mulai offline dari maret dann ...,sekolah ku sudah mulai offline dari maret dan ...,"[sekolah, ku, sudah, mulai, offline, dari, mar...","[sekolah, ku, offline, maret, enak, offline, o...","[sekolah, ku, offline, maret, enak, offline, d...","['sekolah', 'ku', 'offline', 'maret', 'enak', ...",sekolah ku offline maret enak offline daring t...
2189,mana udah offline sekolah,0,mana udah offline sekolah,mana udah offline sekolah,mana sudah offline sekolah,"[mana, sudah, offline, sekolah]","[offline, sekolah]","[offline, sekolah]","['offline', 'sekolah']",offline sekolah
2190,gue kira senin bakal sekolah offline ternyata ...,0,gue kira senin bakal sekolah offline ternyata ...,gue kira senin bakal sekolah offline ternyata ...,gue kira senin bakal sekolah offline ternyata ...,"[gue, kira, senin, bakal, sekolah, offline, te...","[gue, senin, sekolah, offline, online]","[saya, senin, sekolah, offline, daring]","['saya', 'senin', 'sekolah', 'offline', 'daring']",saya senin sekolah offline daring
2191,art! curhat dkit sender akhir2 ini capek bgt s...,1,art curhat dkit sender akhir ini capek bgt stl...,art curhat dkit sender akhir ini capek bgt stl...,art curhat dkit sender akhir ini cape banget s...,"[art, curhat, dkit, sender, akhir, ini, cape, ...","[art, curhat, dkit, sender, cape, banget, seko...","[art, curhat, dkit, sender, capek, banget, sek...","['art', 'curhat', 'dkit', 'sender', 'capek', '...",art curhat dkit sender capek banget sekolah of...


In [22]:
filename = 'NewDataset/dataset-preprocessing.csv'
data_twt.to_csv(filename, columns=['tweet_bersih', 'label_sentimen'], index=False)